<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>


# Working with Storage Resources via the Azure CLI

## Azure Actions Covered

* Creating and deleting storage accounts
* Showing connection strings
* Creating and deleting storage containers under storage accounts
* Uploading, listing, downloading, and deleting blobs

In this lecture, we're going to look at how to interact with Storage accounts and containers in Azure via the CLI. First, make sure you're logged in.

```
$ az login
```

Next, let's create a new resource group using the commands we used in the last section.

```
$ az group create -n azure-storage-group -l eastus
```

Now that a dedicated resource group has been created, we can create a storage account under the resource group using `az storage account create`. **Note:** The storage account name **must** be unique, since it is used in associated URLs! You may have to try a couple to get one that is not in use (hint: try using random numbers).

```
$ az storage account create -g azure-storage-group -l eastus -n benbstorage1
```

We'll need a way to authenticate some of Azure operations when we interact with our Storage resources. The following command shows us our connection string, which is one way of authenticating in the CLI.

```
$ az storage account show-connection-string -g azure-storage-group -n benbstorage1 --query connectionString
```

For security purposes and so that it can be reused later, it's best to store this as a variable. Connection strings can contain information you want to keep private.

```
$ CONNECTION_STRING=$(az storage account show-connection-string -g azure-storage-group -n benbstorage1 --query connectionString)
```

Under our Storage account, we have to also create a container, which is where we can store any files, etc. For this, we'll need the connection string we just created.

```
$ az storage container create --name "bens-blob-container" --account-name benbstorage1 --connection-string $CONNECTION_STRING
```

Great, our storage container has been created! You can use the Azure platform to check that it's where you expect it to be.

Let's make a quick test file to copy into our new container.

```
$ echo "Hello there! I'm a friendly text file" > test_file.txt
```

We can copy the file over to our Storage container without a connection string or account key, but that will give us a warning (since it's not secure).

```
$ az storage copy -s test_file.txt --destination-account-name benbstorage1 --destination-container bens-blob-container
```

Let's add the connection string parameter so that the operation is safe.

```
$ az storage copy -s test_file.txt --account-name benbstorage1 --destination-container bens-blob-container --connection-string $CONNECTION_STRING
```

The operation should have finished successfully. To verify this, use `az storage blob list` to list the contents of the container we've created.

```
$ az storage blob list --account-name benbstorage1 --container-name bens-blob-container --output table
```

Just like we can upload files to the cloud, we can download files from our Storage containers. Let's download our test file and give it a new name.

```
$ az storage blob download --account-name benbstorage1 --container-name bens-blob-container --name test_file.txt --file new_test_file.txt --connection-string $CONNECTION_STRING
```

Deleting files in Storage via the Azure CLI is straightforward as well. Just use the `az storage blob delete` command.

```
$ az storage blob delete --account-name benbstorage1 --container-name bens-blob-container --name test_file.txt --connection-string $CONNECTION_STRING
```

If we list the files in our container, there shouldn't be anything there.

```
$ az storage blob list --account-name benbstorage1 --container-name bens-blob-container
```

Finally, let's clean up our resources so that we keep our cloud environment free and we don't rack up unwanted charges. If you've forgotten the name of your resource group, use `az group list` to list your resource groups.

```
$ az group list
```

Like we did in the last section, use `az group delete` to delete the resource group we created for this lecture.

```
$ az group delete -n azure-storage-group
```